# Select database (i.e. staging, production, local authenticated, local unauthenticated)

In [1]:
import requests
from obi_auth import get_token
from entitysdk.client import Client
from entitysdk.common import ProjectContext
from enum import Enum

class DatabaseEnvironment(Enum):
    STAGING_DATABASE = "staging"
    # PRODUCTION_DATABASE = "production"
    LOCAL_UNAUTHENTICATED = "local_unauthenticated"
    LOCAL_AUTHENTICATED = "local_authenticated"

obi_one_api_url = "http://127.0.0.1:8100"
virtual_lab_id="edb1fb49-660b-477b-af7b-12913b2df7d9"
project_id="d4d004bc-44c5-4f15-94a7-1e4478e82b1d"

database_environment = DatabaseEnvironment.STAGING_DATABASE
# database_environment = DatabaseEnvironment.PRODUCTION_DATABASE
# database_environment = DatabaseEnvironment.LOCAL_UNAUTHENTICATED
# database_environment = DatabaseEnvironment.LOCAL_AUTHENTICATED

if database_environment == DatabaseEnvironment.STAGING_DATABASE:
    """
    - Uncomment in obi-one/.env.run-local before "make run-local":
        export ENTITYCORE_URL=https://staging.openbraininstitute.org/api/entitycore
    """
    entitycore_api_url = "https://staging.openbraininstitute.org/api/entitycore"

# elif database_environment == DatabaseEnvironment.PRODUCTION_DATABASE:
#     """
#     For future: EntityCore not currently in production.
#     - Uncomment in obi-one/.env.run-local before "make run-local":
#         export ENTITYCORE_URL=https://www.openbraininstitute.org/api/entitycore
#     """
#     entitycore_api_url = "https://www.openbraininstitute.org/api/entitycore"

elif DatabaseEnvironment.LOCAL_UNAUTHENTICATED:
    """
    Not yet tested.
    - Launch entitycore locally (make run-local)
    - Add a morphology with a project name (see entitysdk)
    """
    entitycore_api_url = "https://127.0.1:8000/api/entitycore"

elif DatabaseEnvironment.LOCAL_AUTHENTICATED:
    """
    Not yet tested.
    - Change APP_DISABLE_AUTH to False in entitycore/.env.run-local
    - Launch entitycore locally (make run-local)
    - Add a morphology with a project name (see entitysdk morphology.ipynb example)
    """
    entitycore_api_url = "https://127.0.1:8000/api/entitycore"

else:
    raise ValueError(f"Unsupported environment: {database_environment}")


token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=virtual_lab_id, project_id=project_id)
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)

# Fetch a morphology

In [2]:
from entitysdk.models.morphology import (
    ReconstructionMorphology,
)
kwargs = {}
reconstruction_morphologies = client.search_entity(
    entity_type=ReconstructionMorphology, query=kwargs, limit=10
).all()

reconstruction_morphology_id = reconstruction_morphologies[0].id
print(reconstruction_morphology_id)

2cd84e81-87ab-401f-9baf-86abd786aa76


# Call the declared morphology metrics endpoint

In [3]:
# Construct the full endpoint URL
url = f"{obi_one_api_url}/declared/neuron-morphology-metrics/{reconstruction_morphology_id}"

# Set headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Make the GET request
response = requests.get(url, headers=headers)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")


Success: {'aspect_ratio': 0.8852763978166994, 'circularity': 0.8403189319730738, 'length_fraction_above_soma': 0.7324050068855286, 'max_radial_distance': 3919.52734375, 'number_of_neurites': 9, 'soma_radius': 2.759981632232666, 'soma_surface_area': 95.72431182861328}


# Call the generated morphology metrics endpoint

In [4]:
import requests

# Construct the full endpoint URL
url = f"{obi_one_api_url}/generated/morphology-metrics-run-grid"

# Prepare headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Construct request body — adjust this with the actual morphology metrics form data!
# Placeholder example:
request_body = {
  "type": "MorphologyMetricsForm",
  "initialize": {
    "type": "MorphologyMetricsForm.Initialize",
    "morphology": {
      "id_str": str(reconstruction_morphology_id),
    }
  }
}

# Make the POST request
response = requests.post(url, headers=headers, json=request_body)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")

Success: {'0': {'aspect_ratio': 0.8852763978166994, 'circularity': 0.8403189319730738, 'length_fraction_above_soma': 0.7324050068855286, 'max_radial_distance': 3919.52734375, 'number_of_neurites': 9, 'soma_radius': 2.759981632232666, 'soma_surface_area': 95.72431182861328}}


In [5]:
from entitysdk.models.electrical_cell_recording import (
    ElectricalCellRecording,
)
kwargs = {}
Electrical_cell_recording = client.search_entity(
    entity_type=ElectricalCellRecording, query=kwargs, limit=10
).all()

Electrical_cell_recording_id = Electrical_cell_recording[0].id
print(reconstruction_morphology_id)

2cd84e81-87ab-401f-9baf-86abd786aa76


# Call the declared electrophysiology recording metrics endpoint

In [6]:
# Construct the full endpoint URL
url = f"{obi_one_api_url}/declared/electrophysiologyrecording-metrics/{Electrical_cell_recording_id}"

# Set headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Make the GET request
response = requests.get(url, headers=headers)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")


Success: {'brain_region': 'Ventral posteromedial nucleus of the thalamus', 'feature_dict': {'idrest_0': {'spike_count': {'avg': 6.068181818181818, 'num_traces': 88, 'unit': None}, 'time_to_first_spike': {'avg': 74.30350877190865, 'num_traces': 57, 'unit': 'ms'}, 'time_to_last_spike': {'avg': 654.6052631584205, 'num_traces': 57, 'unit': 'ms'}, 'inv_time_to_first_spike': {'avg': 20.44689903535069, 'num_traces': 57, 'unit': 'Hz'}, 'doublet_ISI': {'avg': 51.29490740738937, 'num_traces': 54, 'unit': 'ms'}, 'inv_first_ISI': {'avg': 156.90276701096153, 'num_traces': 54, 'unit': 'Hz'}, 'ISI_log_slope': {'avg': 0.3224772679888807, 'num_traces': 40, 'unit': 'ms'}, 'ISI_CV': {'avg': 0.20563869223058812, 'num_traces': 40, 'unit': None}, 'irregularity_index': {'avg': 16.44670153420433, 'num_traces': 40, 'unit': 'ms'}, 'adaptation_index': {'avg': 0.06856575054792706, 'num_traces': 40, 'unit': None}, 'mean_frequency': {'avg': 39.787800436805405, 'num_traces': 57, 'unit': 'Hz'}, 'strict_burst_number':